
### Analysis of the 2017 Thomas Fire

Author: Tom Gibbens-Matsuyama (tommats00\@github.io)

-   This repository contains the rest of my analysis [github repository](https://github.com/tommats00/eds220-final)

<img src="https://camo.githubusercontent.com/eb96c7e74feb1cad555d796e03bf71e25a4f38058ba619e1e4da5c6a8d86330d/68747470733a2f2f7777772e6c6965666663616272617365722e636f6d2f77702d636f6e74656e742f75706c6f6164732f77696c64666972652d736c6964652d3530302e6a7067" alt="Thomas Fire 2017">

Image Credits: Carsten Shertzer \| Getty Images

# About

### Purpose

In December of 2017, one of California's largest wildfires burned over 280,000 acres of land in Ventura and Santa Barbara counties. The purpose of this analysis is to do the following: visualize how the Air Quality Index (AQI) was effected in Santa Barbara County and produce a false color image containing the Thomas Fire boundary.

### Highlights of analysis:

-   Use `pandas` to read in tabular data
-   Use `rioxarray` and `geopandas` to read in spatial data
-   Read data from a link and local source
-   Filter and transform vector data types with `pandas`
-   Filter and transform spatial data with `rioxarray` and `geopandas`
-   Visualize filtered AQI data
-   Visualize filtered landsat and boundary data

## Dataset descriptions:

#### AQI Data

[AQI data](https://www.airnow.gov/aqi/aqi-basics/) from the [US Environmental Protection Agency (EPA)](https://www.epa.gov/) will be used to visualize the impact of the 2017 Thomas Fire in Santa Barabra County. This data comes from the [pre-generated data files](https://aqs.epa.gov/aqsweb/airdata/download_files.html). These files are updated twice a year, once in June to capture the entire previous year and once in December to capture the Summer.

#### Landsat Collection 2 Level-2 data from Landsat 8

This dataset is a collection of bands from the Landsat Collection 2 Level-2 collected by the Landsat 8 satellite. This data was taken from the [Microsof Planetary Computer data catalogue](https://planetarycomputer.microsoft.com/dataset/landsat-c2-l2) and pre-processed for our use.

#### California Fire Perimeters

The second dataset is a pre-processed geospatial data frame from [Data.gov](https://catalog.data.gov/dataset/california-fire-perimeters-all-b3436). This dataset contains the Thomas Fire perimeter data.

#### References & data sources

AQI Data: Available from: \[(https://aqs.epa.gov/aqsweb/airdata/download_files.html#AQI.)\] Access date: October 23, 2024

Microsoft Planetary Computer Data Catalogue, Landsat collection 2 Level-2. Available from: \[(https://planetarycomputer.microsoft.com/dataset/landsat-c2-l2.)\] Access date: November 19, 2024.

Data.gov Data Catalogue, California Fire Perimeters (all). Available from: \[(https://catalog.data.gov/dataset/california-fire-perimeters-all-b3436.)\] Access date: November 19, 2024.

#### Ackowledgements
Materials created by [Carmen Galaz-Garcia](https://github.com/carmengg) for EDS-220: [Working with Environmental Data](https://meds-eds-220.github.io/MEDS-eds-220-course/)

### Load libraries

In [1]:
#| code-fold: true
#| code-summary: "Show the code"
#| 
# Core library
import pandas as pd 

# For loading in geospatial data
import geopandas as gpd
import rioxarray as rioxr

# For plotting 
import matplotlib.pyplot as plt

### Load data

In [2]:
#| code-fold: true
#| code-summary: "Show the code"

# Load in AQI data for 2017 and 2018 via a link
aqi_17 = pd.read_csv('https://aqs.epa.gov/aqsweb/airdata/daily_aqi_by_county_2017.zip', compression='zip')
aqi_18 = pd.read_csv('https://aqs.epa.gov/aqsweb/airdata/daily_aqi_by_county_2018.zip', compression='zip')

# Load data from the file path
landsat = rioxr.open_rasterio('data/landsat8-2018-01-26-sb-simplified.nc')

# Load in California fires perimeter data, downloaded locally 
perimeters = gpd.read_file('data/California_Fire_Perimeters/California_Fire_Perimeters_(all).shp')

RasterioIOError: data/landsat8-2018-01-26-sb-simplified.nc: No such file or directory

## Santa Barbara county AQI data

#### Preliminary Exploration

When working with any type of data, the first thing we should do is some preliminary exploration. Since the AQI is a tabular dataset, we can use methods such as `head` and `info` for both of our dataframes. This is a great way of viewing our data if you have no other indication of the type of data you are working with.

In [ ]:
# Print head for 2017 AQI
aqi_17.head(3)

In [ ]:
# Print head for 2018 AQI
aqi_18.head(3)

## Data cleaning

Now that we have done some prelimnary exploration of our data, we have an idea of what we are working with. Now, we can clean the data for best practices and filter down to answer our question. Going back to our purpose, we want to visualize the impact of the 2017 Thomas Fire on the AQI in Santa Barbara county. To visualize this question, we want to filter down to columns AQI and Date.

In [ ]:
#| code-fold: true
#| code-summary: "Show the code"

# Use `pd.concat` to combine the two data sets on top of each other
aqi = pd.concat([aqi_17, aqi_18])

# Simplify column names to lower_snake_case
aqi.columns = (aqi.columns
                  .str.lower()
                  .str.replace(' ','_')
                )

# Filter data to only Santa Barbara county
aqi_sb = aqi[aqi['county_name'] == 'Santa Barbara']

# Drop unwanted columns in data frame
aqi_sb = aqi_sb.drop(columns=['state_name', 'county_name', 'state_code', 'county_code', 'number_of_sites_reporting'])

# Change date column to a datetime
aqi_sb.date = pd.to_datetime(aqi_sb['date'])

# Set the index to the data column
aqi_sb = aqi_sb.set_index('date')

## AQI Data Analysis and Visualization

The data is now prepped to be plotted, but our plot would only in how AQI has changed over time. It would be intersting to add another line to our plot to add some more information. We can add a "five day average" by using the `rolling` method. - The `rolling` method computes a moving average - `5D` as our argument indicates a 5 day moving average - Providing `mean` after `rolling` indicates that we want the mean every 5 days

In [ ]:
# Create a five day rolling average column of AQI
aqi_sb['five_day_average'] = aqi_sb['aqi'].rolling('5D').mean()

Our data is now ready to be plotted. Let's plot over AQI and five day average over time

In [ ]:
#| code-fold: true
#| code-summary: "Show the code"

# Initialize empyt figure and axis
fig, ax = plt.subplots()

# Plot a df on the ax axis
aqi_sb.plot(ax=ax)

ax.set_title('AQI value and its five day average from 2017-2018 in Santa Barbara County')
ax.set_xlabel('Date')
ax.set_ylabel('Air Quality Index (AQI) Value')

# Change legend variable names
plt.legend(labels = ['AQI', 'Five Day Average'])

# Display figure
plt.show()

Our plot shows a huge spike in both our AQI and five day average. We know this is from the 2017 Thomas Fire. From [Airnow's](https://www.airnow.gov/aqi/aqi-basics/) AQI, we know that healthy, moderate, and unhealthy values range from 0-50, 51-100, and 150-200 respectively. During the 2017 Thomas Fire we see an AQI value reaching over 300 which is the highest level of concern. Values of 300 or more translate to emergency health conditions, where everyone is more than likely to be affected. 

## Landsat and Fire Perimeter Data

#### Fire Perimeter Data Cleaning

We have already loaded in our California fire perimeter data. We are interested in looking at the geospatial element of the 2017 Thomas Fire. Like before, we want to explore this data and clean it up. The file that we loaded in was a shapefile, so we expect to see a spatial component. Let's take a look.

In [ ]:
# View the variables within our dataframe 
perimeters.head()

In [ ]:
# Check the crs 
perimeters.crs

There are a number of variables in this geospatial dataframe. We are interested in the perimeter for the 2017 Thomas Fire. So, the columns we need are year, fire name, and it's geometry. From our preliminary exploration, we can see that we have all three of those components. Let's clean up the data and then filter down to the 2017 Thomas Fire. Note that we only need to change the column letters from uppercase to lowercase. They already have `_` instead of spaces. After, we can filter down to the 2017 Thomas Fire.

In [ ]:
#| code-fold: true
#| code-summary: "Show the code"

# Select the data frame's columns and lowercase them
perimeters.columns = perimeters.columns.str.lower()

# Select the Thomas Fire Boundary in 2017
thomas_perimeter = perimeters[(perimeters['fire_name'] == 'THOMAS') 
                              & (perimeters['year_'] == 2017)]

### Landsat exploration

The landsat data was read in using `open_rasterio` from the `rioxarray` package. The format of this information is different from the `DataFrame` and `GeoSpatialDataFrame` we have seen in the past. We do not need to use any exploratory methods such as `head` and `info`. All of the information is contained within the array. We just need to call the variable to view it. Let's take a look.

In [ ]:
# View the dataframe
landsat

Notice, there are 4 coordinates, consisting of a `band`, `x`, `y`, and `spatial_ref`. There are 5 data variables `red`, `green`, `blue`, `nir08`, and `swir22` that represent band types. There are 3 dimensions `band`, `x`, and `y`. There is also an index consisting of `band`, `x`, and `y`. There are options to show/hide the information contained within the coordinates and variables.

## Landsat cleaning

Notice that there is only a value of one in the band dimension. Since there is only one band, we do not need to specify it as a dimension. We can filter this out from our array using the following methods.

In [ ]:
#| code-fold: true
#| code-summary: "Show the code"

# Drop the `band` coordinate of the data
landsat = landsat.drop_vars('band')

# Drop the `band` dimension of the data
# The code `squeeze` drops dimensions with values of 1 
landsat = landsat.squeeze()

# Check our array to confirm that `band` has been 
# removed from the dimensions and coordinates
landsat

Whenever you are working with spatial data, you always want the CRSs to be the same. Otherwise, the spatial data won't be able to communicate properly with one another. We know that the Thomas Fire is projected to EPSG:3857. Let's reproject our landsat data to the same CRS.

In [ ]:
#| code-fold: true
#| code-summary: "Show the code"

# Reproject our landsat data to EPSG:3857
landsat = landsat.rio.reproject("EPSG:3857")

# Check to see if the CRSs are the same
# By using assert we shouldn't see an output
assert landsat.rio.crs == thomas_perimeter.crs

### Let's create our false color image now

To do so we need to change the type of bands that we are plotting. We want to select the short-wave infrared (swir22), near-infrared (nir08), and red bands to the red, green, and blue colors respectively. This will give us our false color image.

In [ ]:
#| code-fold: true
#| code-summary: "Show the code"

# Select swir22, nir08, and red as the variables to be plotted 
landsat[['swir22', 'nir08', 'red']].to_array().plot.imshow(robust = True)

## Step 6: Add the Thomas Fire perimeter to our false color image

We want to combine our Thomas Fire perimeter with our landsat plot to help us visualize where the fire boundary is. The CRSs are both `EPSG:3857` so they should be able to plot with one another without any problems. The first thing we need to do is assign a variable name to our false color image that we created above. Then, we can plot both maps together.

Let's use the Thomas Fire perimeter that we created in the other notebook and add it to the false color image. This will give us an indication of where exactly to look when viewing the map. Remember, when working with spatial data, it is imperative to use the same CRS when comparing data. We know from before that our Thomas Fire data has a CRS of `EPSG:3857`, so let's use this as our base CRS.

In [ ]:
#| code-fold: true
#| code-summary: "Show the code"

# Create a variable for the false color image map
false_color = landsat[['swir22', 'nir08', 'red']].to_array()

# Set up fig and ax
fig, ax = plt.subplots()

# False color image map
false_color.plot.imshow(robust = True, 
                        ax = ax,
                       alpha = 1)

# Fire perimeters
thomas_perimeter.boundary.plot(ax=ax,
                              color = 'darkblue')
                             
# Add a legend for the fire perimeter
ax.legend(labels = ['Thomas Fire Perimeter'])

# Add a title 
plt.title("Area burned from the 2017 Thomas Fire in Ventura and Santa Barbara County")
ax.axis('off')

# Add footnote citation at the bottom of the figure
fig.text(0.5, 0.09, 
         'Data Source: CAL FIRE via Data.gov &  Microsoft Planetary Computer data catalogue',
         ha='center', va='center', fontsize=8, color='black', fontstyle='italic')

fig.text(0.395, 0.06, 
         'Date Accessed: 11/19/24',
         ha='right', va='center', fontsize=8, color='black', fontstyle='italic')


plt.show()

This false color image does a great job of identifying the burned area from the 2017 Thomas Fire, as indicated in red. Having the Thomas Fire boundary layered over top of the image depicts the burned area even more. Remote sensing has become an important part of detecting, monitoring, and acting on wildfire outbreaks. This is just one example of the power that can be harnessed from satellite data.

This workflow depicts the area burned and the dramatic increase of PM 2.5 in air quality during the 2017 Thomas Fire.